In [12]:
%reload_ext dotenv
%dotenv
config_file_path='./config-dataunion-mumbai.ini'
import ocean_lib
import sys
import time
import os
from ocean_lib.config import Config
from ocean_lib.ocean.ocean import Ocean
from datetime import datetime
from ocean_lib.web3_internal.currency import to_wei
from ocean_lib.web3_internal.wallet import Wallet
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
from ocean_lib.services.service import Service
from ocean_lib.assets.asset_resolver import resolve_asset
from ocean_lib.common.agreements.service_types import ServiceTypes
from ocean_lib.assets.trusted_algorithms import add_publisher_trusted_algorithm

print(f"Using python:{sys.version}")
print(f"Using ocean_lib: {ocean_lib.__version__}")

Using python:3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]
Using ocean_lib: 0.8.5


In [19]:
config = Config(config_file_path)
ocean = Ocean(config)
provider_url = DataServiceProvider.get_url(ocean.config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

# Create Alice's wallet
alice_wallet = Wallet(
    ocean.web3,
    os.getenv('PRIVATE_KEY1'),
    config.block_confirmations,
    config.transaction_timeout,
)

bob_wallet = Wallet(
    ocean.web3,
    os.getenv('PRIVATE_KEY2'),
    config.block_confirmations,
    config.transaction_timeout,
)
print(f"wallet.address = '{alice_wallet.address}'")

config.network_url = 'https://rpc-mumbai.maticvigil.com'
config.block_confirmations = 1
config.metadata_cache_uri = 'https://aquarius.dataunion.market'
config.provider_url = 'https://provider.dev.dataunion.market/'
wallet.address = '0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA'


In [20]:
DATA_did = 'did:op:4508E13742e8B3f105033498a9299866b0688488'  # for convenience
ALG_did = 'did:op:9196BFBC51AD7617161b9a8ea867eDb5aECAc1F5'
DATA_DDO = ocean.assets.resolve(DATA_did)  # make sure we operate on the updated and indexed metadata_cache_uri versions
ALG_DDO = ocean.assets.resolve(ALG_did)

In [22]:
compute_service = DATA_DDO.get_service('compute')
algo_service = ALG_DDO.get_service('access')

from ocean_lib.web3_internal.constants import ZERO_ADDRESS
from ocean_lib.models.compute_input import ComputeInput

# order & pay for dataset
dataset_order_requirements = ocean.assets.order(
    DATA_did, bob_wallet.address, service_type=compute_service.type
)
DATA_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        dataset_order_requirements.amount,
        dataset_order_requirements.data_token_address,
        DATA_did,
        compute_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        dataset_order_requirements.computeAddress,
    )

# order & pay for algo
algo_order_requirements = ocean.assets.order(
    ALG_did, bob_wallet.address, service_type=algo_service.type
)
ALG_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        algo_order_requirements.amount,
        algo_order_requirements.data_token_address,
        ALG_did,
        algo_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        algo_order_requirements.computeAddress,
)

custom_data = {
    'id': 'TdQovMPnnWPtvKE'
}

compute_inputs = [ComputeInput(DATA_did, DATA_order_tx_id, compute_service.index)]
job_id = ocean.compute.start(
    compute_inputs,
    bob_wallet,
    algorithm_did=ALG_did,
    algorithm_tx_id=ALG_order_tx_id,
    algorithm_data_token=ALG_DDO.address,
    algouserdata=custom_data
)
print(f"Started compute job with id: {job_id}")

AttributeError: 'V3Asset' object has no attribute 'address'

In [ ]:
ocean.compute.status(DATA_did, job_id, bob_wallet)

In [ ]:
result = ocean.compute.result_file(DATA_did, job_id, 1, bob_wallet)  # 0 index, means we retrieve the results from the first dataset index
print(result)